Импортируем необходимые библиотеки.

In [ ]:
# Клонирование репозитория StyleGAN2
!git clone https://github.com/rosinality/stylegan2-pytorch.git
import os

import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import transforms
from torchvision.utils import save_image

import clip

from PIL import Image

import numpy as np

import matplotlib.pyplot as plt

In [ ]:
!pip install google.colab
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# downloads StyleGAN-NADA's weights
ids = ['1bhWgbI7oleIBPs9kE7IL7LGkZpzNOsFO', '1EM87UquaoQmk17Q8d5kYIAHqu0dkYqdT']
for file_id in ids:
  downloaded = drive.CreateFile({'id':file_id})
  downloaded.FetchMetadata(fetch_all=True)
  downloaded.GetContentFile(downloaded.metadata['title'])

In [ ]:
def visual_compare(filtered, clean):
    fig, axs = plt.subplots(1, 2, figsize=(16, 32))  # Создание подграфиков для визуализации
    # Визуализация сгенерированного изображения
    axs[1].imshow((filtered.cpu().detach()[0].clip(-1,1).permute(1, 2, 0) + 1) / 2)
    axs[1].set_title("Обработанное изображение StyleGAN-NADA")  # Заголовок для сгенерированного изображения
    axs[1].axis('off')  # Отключение осей

    # Визуализация целевого изображения
    axs[0].imshow((clean.cpu().detach()[0].clip(-1,1).permute(1, 2, 0) + 1) / 2)
    axs[0].set_title("Исходное изображение")  # Заголовок для исходного изображения
    axs[0].axis('off')  # Отключение осей
    plt.show()

def load_image(img_path, size=1024):
    img = Image.open(img_path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize(size),
        transforms.CenterCrop(size),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
    return transform(img).unsqueeze(0).to(device)


def invert_image(target_img, generator, steps=500, lr=0.1, truncation=0.7):
    # Инициализация случайного латентного вектора
    z = torch.randn(1, 512, device=device)  
    w = generator.get_latent(z).detach().clone().requires_grad_(True)
    
    optimizer = torch.optim.Adam([w], lr=lr)
    loss_fn_mse = torch.nn.MSELoss()
    loss_fn_vgg = lpips.LPIPS(net="vgg").to(device)  # Perceptual loss

    for _ in range(steps):
        # Генерация изображения из текущего w
        generated_img, _ = generator([w], input_is_latent=True, truncation=truncation)
        
        # Вычисление потерь (MSE + LPIPS)
        loss_mse = loss_fn_mse(generated_img, target_img)
        loss_lpips = loss_fn_vgg(generated_img, target_img).mean()
        loss = loss_mse + loss_lpips
        
        # Оптимизация
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return w.detach()

In [ ]:
generator = Generator(size=1024, style_dim=latent_dim, n_mlp=8).to(device)
state_dict_nada = torch.load('weights.pth', map_location=device)
generator.load_state_dict(state_dict_nada['g_ema'])
generator.eval()

In [ ]:
generator_clean = Generator(size=1024, style_dim=latent_dim, n_mlp=8).to(device)
state_dict_nada = torch.load('stylegan2-ffhq-config-f.pt', map_location=device)
generator_clean.load_state_dict(state_dict['g_ema'])
generator_clean.eval()

In [ ]:
z = torch.randn(1, 512).to(device)
w = generator.get_latent(z)
image, _ = generator([w], input_is_latent=True, randomize_noise=False)
image_clean, _ = generator_clean([w], input_is_latent=True, randomize_noise=False)

In [ ]:
visual_compare(image, image_clean)